# Establecemos las librerías a utilizarse

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargamos la base de Datos para su análisis

Nota: No permitía establecer la conexión en snowflake, por lo que se procedió a cargar el CSV directamente desde el cordenador.

In [2]:
raw_diesel = pd.read_csv(r"C:\Users\andre\OneDrive\Escritorio\USFQ\Fundamentos en Ciencia de Datos\Proyeccion_Consumo_Diesel\dataset\Consumo_Diesel.csv", sep=";")
raw_diesel.head()

,FECHA,DISPENSADOR,HORA,VEHICULO,FLOTA,GALONES,KILOMETRAJE
0,1/1/2020,1,10:35:00,S024,B12M,"53,5680","411475,8000"
1,1/1/2020,1,12:12:00,S029,B12M,"54,1970","479353,4000"
2,1/1/2020,1,12:20:00,S011,B12M,"56,5560","463696,8000"
3,1/1/2020,1,13:10:00,S074,B12M,"65,6140","451279,5000"
4,1/1/2020,1,13:30:00,S015,B12M,"42,8790","418781,5000"


In [3]:
raw_diesel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228209 entries, 0 to 228208
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   FECHA        228209 non-null  object
 1   DISPENSADOR  228209 non-null  int64 
 2   HORA         228209 non-null  object
 3   VEHICULO     228207 non-null  object
 4   FLOTA        193574 non-null  object
 5   GALONES      228209 non-null  object
 6   KILOMETRAJE  227988 non-null  object
dtypes: int64(1), object(6)
memory usage: 12.2+ MB


Se puede observar que existe un total de 228209 datos ingresados, y que en las columnas pertenecientes a "VEHICULO", "FLOTA", "GALONES" y "KILOMERAJE" presentan valores nulos, también se puede observar que en la "FECHA", "HORA", "GALONES" y "KILOMETRAJE" no tienen los formatos adecuados, ya que estas se almacenan como texto y no como tiempo/fecha y valores flotantes.

# Valores Duplicados

In [4]:
raw_duplicados = raw_diesel.duplicated().sum()
print('Se encontraron valores duplicados: ', raw_duplicados)

Se encontraron valores duplicados:  1


In [5]:
print('El dato duplicado es el siguiente: ')
raw_diesel[raw_diesel.duplicated()]

El dato duplicado es el siguiente: 


,FECHA,DISPENSADOR,HORA,VEHICULO,FLOTA,GALONES,KILOMETRAJE
23196,3/8/2020,4,10:15:00,T008,TROLEBUS,"2,8310","63703,0000"


In [6]:
raw_diesel.query("FECHA == '3/8/2020' & HORA == '10:15:00' & VEHICULO == 'T008' & GALONES == '2,8310'")

,FECHA,DISPENSADOR,HORA,VEHICULO,FLOTA,GALONES,KILOMETRAJE
23195,3/8/2020,4,10:15:00,T008,TROLEBUS,"2,8310","63703,0000"
23196,3/8/2020,4,10:15:00,T008,TROLEBUS,"2,8310","63703,0000"


# Valores Faltantes

In [7]:
raw_diesel.isna().sum()

FECHA              0
DISPENSADOR        0
HORA               0
VEHICULO           2
FLOTA          34635
GALONES            0
KILOMETRAJE      221
dtype: int64

In [8]:
ausentes_vh = raw_diesel[raw_diesel.isna()['VEHICULO']]
ausentes_vh

,FECHA,DISPENSADOR,HORA,VEHICULO,FLOTA,GALONES,KILOMETRAJE
147742,1/8/2022,6,0:00:00,NaN,NaN,"0,0000","0,0000"
172389,11/12/2022,4,18:29:00,NaN,NaN,"0,0010","0,0000"


Como se puede observa en la columna "VEHICULO" se encuentran datos vacíos, y tomando en cuenta la fila de "GALONES" y "KILOMETRAJE" pueden ser errores en los registros, los cuales al ser eliminados no afectan al estudio ya que los valores que registran tanto en galonaje como kilometraje son mínimos.

In [9]:
ausentes_flt = raw_diesel[raw_diesel.isna()['FLOTA']]
ausentes_flt['VEHICULO'].unique()

array(['M036', 'M026', 'D015', 'D003', 'D025', 'T111', 'M020', 'D009',
       'D030', 'D042', 'D002', 'T095', 'T033', 'M019', 'S056', 'B036',
       'B038', 'T051', 'A114', 'B045', 'T028', 'T042', 'T005', 'T043',
       'T003', 'D029', 'D006', 'T038', 'T029', 'D038', 'M040', 'T032',
       'D020', 't029', 't111', 'T81', 'D004', 'D016', 'T045', 'D024',
       'T048', 'D021', 'T019', 'T053', 'D034', 'T075', 'D033', 'T031',
       'D010', 'A117', 'T088', 'T015', 'T037', 'COMPRESOR', 'T034',
       'B048', 'D039', 'D040', 'T030', 'T102', 'D011', 'PMA8561',
       'PMA7709', 'PMA7706', 'PMF780', 'PMA7390', 'PMA8560', 'PMA7727',
       'PMA7715', 'PMA8565', 'PMA3026', 'PMA7585', 'PMA3244', 'PMA3325',
       'PMA8566', 'PMA7158', 'PMA7728', 'PMA7393', 'PMA8567', 'PMA8557',
       'PMA8568', 'PMA7707', 'PMA7713', 'PMA8559', 'PMA8562', 'PMA7729',
       'PMA7718', 'PMA7712', 'PMA7708', 'PMA7719', 'PMA8570', 'PMA8564',
       'PMA7561', 'PMA3083', 'PMA7189', 'PMA7714', 'PMA7721', 'PMA3065',
    

Se puede obervar que existen varios valores que no describen un tipo de flota, en su mayoría son errores al momento de registrar las placas de los vehículos, como la "t029" que en el registro real si existe una "T029", también se puede observar que existen el registro de unidades que empiezan con una nomenclatura "PMA" o "P", esto se presenta debido a que no son unidades pertenecientes a la flota Trolebus. También se puede observar que existen unidades que han sido dadas de baja en tiempo recientes, por lo que no se encuentran registrados en un flota en específico. 

In [10]:
ausentes_km = raw_diesel[raw_diesel.isna()['KILOMETRAJE']]
ausentes_km['VEHICULO'].unique()

array(['LAVTALLER', 'T073', 'D006', 'M038', 'COMPRESOR', 'A114', 'B032',
       'M014', 'T106', 'T017', 'T004', 'PMA3053', 'PMA3026', 'PMA7672',
       'PMA3322', 'PMA3249', 'PMA3066', 'PMF780', 'PMA3325', 'PMA8561',
       'PMA706', 'P-600789561', 'PMA7388', 'PMA3050', 'PMA7394',
       'PMA3039', 'PMA7717', 'PMA3267', 'D035', 'PMA7721', 'D037',
       'P-G00789561', 'PMA3237', 'PMA7230', 'PMA8558', 'lavtaller',
       'PMA7684', 'PMF0782', 'M011', 'B052', 'PMA7164', 'PMA3093',
       'PMF0872', 'PMA7561', 'PME0782', 'PMA8567', 'PMA7113', 'PMA7689',
       'PMA7878', 'PMA3239', 'PMA7108', 'PMA7699', 'PMD0501', 'P-30162',
       'PMA7032', 'PMA7713', 'PMA8564', 'PMA7109', 'S008', 'PMA7393',
       'V017', 'V060', 'T027'], dtype=object)

Los datos de "COMPRESOR" y "LAVTALLER" son implementos de trabajo en los que se utiliza diesel y no tienen recorrido en kilómetros, las unidades que empiezan con "P" o "PMA" son unidades de trabajo no pertenecientes a la empresa por lo que no se registra el kilometraje, las unidades "T", "D", "M", "B" y "A", se completarán los datos con la media tel recorrido de la unidades y el último registro.